In [ ]:
from google.colab import files
uploaded = files.upload()

Saving tourism_rating.csv to tourism_rating.csv
Saving dummy_user.csv to dummy_user.csv
Saving tourism_with_id.csv to tourism_with_id.csv


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

rating = pd.read_csv('tourism_rating.csv', sep=',', error_bad_lines=False, encoding="latin-1")
user = pd.read_csv('dummy_user.csv', sep=',', error_bad_lines=False, encoding="latin-1")
tourism = pd.read_csv('tourism_with_id.csv', sep=',', error_bad_lines=False, encoding="latin-1")

In [ ]:
tourism

,Place_Id,Place_Name,Category,City,Price,Rating,Time_Minutes
0,1,Monas,Budaya,Jakarta,20000,4.6,15.0
1,2,Kota Tua,Budaya,Jakarta,0,4.6,90.0
2,3,Dunia Fantasi,Taman Hiburan,Jakarta,270000,4.6,360.0
3,4,Taman Mini Indonesia Indah (TMII),Taman Hiburan,Jakarta,10000,4.5,NaN
4,5,Atlantis Water Adventure,Taman Hiburan,Jakarta,94000,4.5,60.0
...,...,...,...,...,...,...,...
444,445,Museum Mpu Tantular,Budaya,Surabaya,2000,4.4,45.0
445,446,Taman Bungkul,Taman Hiburan,Surabaya,0,4.6,NaN
446,447,Taman Air Mancur Menari Kenjeran,Taman Hiburan,Surabaya,0,4.4,45.0
447,448,Taman Flora Bratang Surabaya,Taman Hiburan,Surabaya,0,4.6,NaN


In [ ]:
rating

,User_Id,Place_Id,Place_Ratings
0,1,322,3
1,1,304,2
2,1,171,5
3,1,407,3
4,1,270,4
...,...,...,...
9995,300,427,2
9996,300,186,4
9997,300,237,3
9998,300,244,4


In [ ]:
user

,User_Id,Location,Age
0,1,"Semarang, Jawa Tengah",20
1,2,"Bekasi, Jawa Barat",21
2,3,"Cirebon, Jawa Barat",23
3,4,"Bekasi, Jawa Barat",21
4,5,"Lampung, Sumatera Selatan",20
...,...,...,...
295,296,"Lampung, Sumatera Selatan",31
296,297,"Palembang, Sumatera Selatan",39
297,298,"Bogor, Jawa Barat",38
298,299,"Sragen, Jawa Tengah",27


In [ ]:
tourism.isna().sum()

Place_Id          0
Place_Name        0
Category          0
City              0
Price             0
Rating            0
Time_Minutes    236
dtype: int64

In [ ]:
tourism[tourism.isna().any(axis=1)]

,Place_Id,Place_Name,Category,City,Price,Rating,Time_Minutes
3,4,Taman Mini Indonesia Indah (TMII),Taman Hiburan,Jakarta,10000,4.5,NaN
6,7,Kebun Binatang Ragunan,Cagar Alam,Jakarta,4000,4.5,NaN
7,8,Ocean Ecopark,Taman Hiburan,Jakarta,180000,4.0,NaN
8,9,Pelabuhan Marina,Bahari,Jakarta,175000,4.4,NaN
9,10,Pulau Tidung,Bahari,Jakarta,150000,4.5,NaN
...,...,...,...,...,...,...,...
438,439,Patung Buddha Empat Rupa,Budaya,Surabaya,0,4.3,NaN
442,443,Taman Hiburan Rakyat,Taman Hiburan,Surabaya,5000,4.2,NaN
445,446,Taman Bungkul,Taman Hiburan,Surabaya,0,4.6,NaN
447,448,Taman Flora Bratang Surabaya,Taman Hiburan,Surabaya,0,4.6,NaN


In [ ]:
tourism.mean()

Place_Id          225.000000
Price           24374.164811
Rating              4.443430
Time_Minutes       82.746479
dtype: float64

In [ ]:
tourism.fillna(tourism.mean(),inplace=True)

In [ ]:
tourism.isna().sum()

Place_Id        0
Place_Name      0
Category        0
City            0
Price           0
Rating          0
Time_Minutes    0
dtype: int64

In [ ]:
tourism_rating = pd.merge(rating, tourism, on='Place_Id')
cols = ['Category', 'Price', 'City', 'Rating', 'Time_Minutes']
tourism_rating.drop(cols, axis=1, inplace=True)


In [ ]:
tourism_rating

,User_Id,Place_Id,Place_Ratings,Place_Name
0,1,322,3,Taman Begonia
1,17,322,2,Taman Begonia
2,39,322,3,Taman Begonia
3,98,322,3,Taman Begonia
4,103,322,4,Taman Begonia
...,...,...,...,...
9995,229,366,2,Saloka Theme Park
9996,233,366,5,Saloka Theme Park
9997,233,366,3,Saloka Theme Park
9998,281,366,3,Saloka Theme Park


In [ ]:
rating_count = (tourism_rating.
     groupby(by = ['Place_Name'])['Place_Ratings'].
     count().
     reset_index().
     rename(columns = {'Place_Ratings': 'RatingCount_Tourism'})
     [['Place_Name', 'RatingCount_Tourism']]
    )
    
threshold = 15
rating_count = rating_count.query('RatingCount_Tourism >= @threshold')

user_rating = pd.merge(rating_count, tourism_rating, left_on='Place_Name', right_on='Place_Name', how='left')



In [ ]:
user_rating

,Place_Name,RatingCount_Tourism,User_Id,Place_Id,Place_Ratings
0,Ade Irma Suryani Nasution Traffic Park,31,5,289,2
1,Ade Irma Suryani Nasution Traffic Park,31,10,289,3
2,Ade Irma Suryani Nasution Traffic Park,31,14,289,2
3,Ade Irma Suryani Nasution Traffic Park,31,27,289,2
4,Ade Irma Suryani Nasution Traffic Park,31,31,289,3
...,...,...,...,...,...
9721,Wot Batu,18,203,270,5
9722,Wot Batu,18,204,270,3
9723,Wot Batu,18,263,270,5
9724,Wot Batu,18,271,270,5


In [ ]:
user_count = (user_rating.
     groupby(by = ['User_Id'])['Place_Ratings'].
     count().
     reset_index().
     rename(columns = {'Place_Ratings': 'RatingCount_user'})
     [['User_Id', 'RatingCount_user']]
    )
    
threshold = 10
user_count = user_count.query('RatingCount_user >= @threshold')

combined = user_rating.merge(user_count, left_on = 'User_Id', right_on = 'User_Id', how = 'inner')



In [ ]:
combined

,Place_Name,RatingCount_Tourism,User_Id,Place_Id,Place_Ratings,RatingCount_user
0,Ade Irma Suryani Nasution Traffic Park,31,5,289,2,33
1,Air Terjun Kedung Pedut,25,5,134,4,33
2,Alun Alun Selatan Yogyakarta,18,5,96,2,33
3,Bumi Perkemahan Cibubur,21,5,37,2,33
4,Candi Prambanan,21,5,122,3,33
...,...,...,...,...,...,...
9721,Taman Pelangi Yogyakarta,31,254,98,4,24
9722,Watu Mabur Mangunan,26,254,166,2,24
9723,Wisata Eling Bening,19,254,382,3,24
9724,Wisata Kalibiru,20,254,188,4,24


In [ ]:
print('Number of unique tourism: ', combined['Place_Name'].nunique())
print('Number of unique users: ', combined['User_Id'].nunique())

Number of unique tourism:  428
Number of unique users:  300


In [ ]:
scaler = MinMaxScaler()
combined['Place_Ratings'] = combined['Place_Ratings'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(combined['Place_Ratings'].values.reshape(-1,1)))
combined['Place_Ratings'] = rating_scaled

In [ ]:
combined = combined.drop_duplicates(['User_Id', 'Place_Name'])
user_tourism_matrix = combined.pivot(index='User_Id', columns='Place_Name', values='Place_Ratings')
user_tourism_matrix.fillna(0, inplace=True)

users = user_tourism_matrix.index.tolist()
tourisms = user_tourism_matrix.columns.tolist()

user_tourism_matrix = user_tourism_matrix.values

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
num_input = combined['Place_Name'].nunique()
num_hidden_1 = 10
num_hidden_2 = 5

X = tf.placeholder(tf.float64, [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1], dtype=tf.float64)),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2], dtype=tf.float64)),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1], dtype=tf.float64)),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input], dtype=tf.float64)),
}

biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2], dtype=tf.float64)),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'decoder_b2': tf.Variable(tf.random_normal([num_input], dtype=tf.float64)),
}

In [ ]:
def encoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']), biases['encoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))
    return layer_2

def decoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']), biases['decoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']), biases['decoder_b2']))
    return layer_2

In [ ]:

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

y_pred = decoder_op

y_true = X

In [ ]:
loss = tf.losses.mean_squared_error(y_true, y_pred)
optimizer = tf.train.RMSPropOptimizer(0.03).minimize(loss)
eval_x = tf.placeholder(tf.int32, )
eval_y = tf.placeholder(tf.int32, )
pre, pre_op = tf.metrics.precision(labels=eval_x, predictions=eval_y)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
init = tf.global_variables_initializer()
local_init = tf.local_variables_initializer()
pred_data = pd.DataFrame()

In [ ]:
with tf.Session() as session:
    epochs = 100
    batch_size = 35

    session.run(init)
    session.run(local_init)

    num_batches = int(user_tourism_matrix.shape[0] / batch_size)
    user_tourism_matrix = np.array_split(user_tourism_matrix, num_batches)
    
    for i in range(epochs):

        avg_cost = 0
        for batch in user_tourism_matrix:
            _, l = session.run([optimizer, loss], feed_dict={X: batch})
            avg_cost += l

        avg_cost /= num_batches

        print("epoch: {} Loss: {}".format(i + 1, avg_cost))

    user_tourism_matrix = np.concatenate(user_tourism_matrix, axis=0)

    preds = session.run(decoder_op, feed_dict={X: user_tourism_matrix})

    pred_data = pred_data.append(pd.DataFrame(preds))

    pred_data = pred_data.stack().reset_index(name='Place_Ratings')
    pred_data.columns = ['User_Id', 'Place_Name', 'Place_Ratings']
    pred_data['User_Id'] = pred_data['User_Id'].map(lambda value: users[value])
    pred_data['Place_Name'] = pred_data['Place_Name'].map(lambda value: tourisms[value])
    
    keys = ['User_Id', 'Place_Name']
    index_1 = pred_data.set_index(keys).index
    index_2 = combined.set_index(keys).index

    top_ten_ranked = pred_data[~index_1.isin(index_2)]
    top_ten_ranked = top_ten_ranked.sort_values(['User_Id', 'Place_Ratings'], ascending=[True, False])
    top_ten_ranked = top_ten_ranked.groupby('User_Id').head(10)

epoch: 1 Loss: 0.3498840741813183
epoch: 2 Loss: 0.3497105985879898
epoch: 3 Loss: 0.34944650158286095
epoch: 4 Loss: 0.34904467687010765
epoch: 5 Loss: 0.34843404963612556
epoch: 6 Loss: 0.3475074991583824
epoch: 7 Loss: 0.3461056165397167
epoch: 8 Loss: 0.3439946360886097
epoch: 9 Loss: 0.3408418446779251
epoch: 10 Loss: 0.33619847521185875
epoch: 11 Loss: 0.32950766757130623
epoch: 12 Loss: 0.32013222947716713
epoch: 13 Loss: 0.3073381744325161
epoch: 14 Loss: 0.2903631813824177
epoch: 15 Loss: 0.2687258757650852
epoch: 16 Loss: 0.241294726729393
epoch: 17 Loss: 0.20608869194984436
epoch: 18 Loss: 0.16477215848863125
epoch: 19 Loss: 0.1240511042997241
epoch: 20 Loss: 0.08898668084293604
epoch: 21 Loss: 0.06409234972670674
epoch: 22 Loss: 0.047530374489724636
epoch: 23 Loss: 0.038201368879526854
epoch: 24 Loss: 0.031311576487496495
epoch: 25 Loss: 0.027696311939507723
epoch: 26 Loss: 0.026995052117854357
epoch: 27 Loss: 0.026836553821340203
epoch: 28 Loss: 0.02678107679821551
epoch: 

In [ ]:
top_ten_ranked.loc[top_ten_ranked['User_Id'] == 19]

,User_Id,Place_Name,Place_Ratings
7851,19,Kawasan Wisata Sosrowijayan,0.094833
7850,19,Kawasan Punclut,0.089913
8095,19,Tebing Breksi,0.078591
7825,19,Jembatan Pasupati,0.078542
8045,19,Taman Air Mancur Menari Kenjeran,0.077138
8030,19,Setu Babakan,0.076923
7853,19,Kebun Binatang Bandung,0.076503
7979,19,Pantai Sundak,0.074481
8115,19,Watu Goyang,0.074067
7891,19,Monumen Batik Yogyakarta,0.073170


In [ ]:
tourism_rating.loc[tourism_rating['User_Id'] ==16].sort_values(by=['Place_Ratings'], ascending=False)

,User_Id,Place_Id,Place_Ratings,Place_Name
258,16,213,5,Desa Wisata Tembi
425,16,214,5,Wisata Kaliurang
6433,16,218,5,Pantai Congot
5060,16,342,5,Rainbow Garden
6511,16,312,5,Masjid Al-Imtizaj
4728,16,435,4,Patung Sura dan Buaya
2288,16,317,4,Pemandian Air Panas Ciater
2755,16,12,4,Pulau Pari
6480,16,72,4,Waterboom PIK (Pantai Indah Kapuk)
3216,16,247,4,Lembang Park & Zoo
